In [16]:
import cv2
import itertools
import numpy as np
import mediapipe as mp
import pandas as pd

In [32]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    refine_landmarks=True,
    static_image_mode=True,
    max_num_faces = 1,
)

mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

for i in range(100):
    image = cv2.imread("./dataset/train/img/%s.png"%i)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    globals()['height'+str(i)] = image.shape[0]
    
    results = face_mesh.process(image)
    
    for single_face_landmarks in results.multi_face_landmarks:
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list = single_face_landmarks,
            connections=mp_face_mesh.FACEMESH_CONTOURS,
            landmark_drawing_spec = drawing_spec,            
        )
    
    INDEX = mp_face_mesh.FACEMESH_NOSE
    INDEX_LIST = list(itertools.chain(*INDEX))
    INDEX_LIST = list(set(INDEX_LIST))
    
    globals()['facemesh'+str(i)] = np.zeros((len(INDEX_LIST), 2))
    
    for single_face_landmarks in results.multi_face_landmarks:
        for j, k in enumerate(INDEX_LIST):
            coordinates = single_face_landmarks.landmark[k]
            globals()['facemesh'+str(i)][j][0] = coordinates.x * image.shape[1]
            globals()['facemesh'+str(i)][j][1] = coordinates.y * image.shape[0]

In [45]:
for i in range(100):
    land = np.zeros((len(facemesh0),2))
    
    for j in range(len(facemesh0)):
        land[j][0] = globals()['facemesh'+str(i)][j][0]
        land[j][1] = globals()['facemesh'+str(i)][j][1]
        
    x = land[:, 0]
    y = land[:, 1] * (-1) + globals()['height'+str(i)]
    
    x -= ((min(x) + max(x)) / 2)
    y -= ((min(y) + max(y)) / 2)
    
    norm_x = (x - min(x)) / (max(y)-min(y))
    norm_y = (y - min(y)) / (max(y)-min(y))
    
    globals()['norm'+str(i)] = np.column_stack((norm_x, norm_y))
    
    globals()['norm'+str(i)] = globals()['norm'+str(i)].flatten()

In [35]:
train_x = np.zeros((100, len(norm0)))
for i in range(100):
    for j in range(len(norm0)):
        train_x[i][j] = globals()['norm'+str(i)][j]

In [36]:
for i in range(100):
    image = cv2.imread("./dataset/test/img/%s.png"%i)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    globals()['height'+str(i)] = image.shape[0]
    
    results = face_mesh.process(image)
    
    for single_face_landmark in results.multi_face_landmarks:
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list = single_face_landmarks,
            connections=mp_face_mesh.FACEMESH_CONTOURS,
            landmark_drawing_spec = drawing_spec,            
        )
    
    INDEX = mp_face_mesh.FACEMESH_NOSE
    INDEX_LIST = list(itertools.chain(*INDEX))
    INDEX_LIST = list(set(INDEX_LIST))
    
    globals()['facemesh'+str(i)] = np.zeros((len(INDEX_LIST), 2))
    
    for single_face_landmarks in results.multi_face_landmarks:
        for j, k in enumerate(INDEX_LIST):
            coordinates = single_face_landmarks.landmark[k]
            globals()['facemesh'+str(i)][j][0] = coordinates.x * image.shape[1]
            globals()['facemesh'+str(i)][j][1] = coordinates.y * image.shape[0]

In [21]:
print(set(INDEX_LIST))

{1, 2, 4, 5, 6, 19, 275, 278, 294, 168, 45, 48, 440, 64, 195, 197, 326, 327, 344, 220, 94, 97, 98, 115}


In [37]:
for i in range(100):
    land = np.zeros((len(facemesh0),2))
    
    for j in range(len(facemesh0)):
        land[j][0] = globals()['facemesh'+str(i)][j][0]
        land[j][1] = globals()['facemesh'+str(i)][j][1]
        
    x = land[:, 0]
    y = land[:, 1] * (-1) + globals()['height'+str(i)]
    
    x -= ((min(x) + max(x)) / 2)
    y -= ((min(y) + max(y)) / 2)
    
    norm_x = (x - min(x)) / (max(y)-min(y))
    norm_y = (y - min(y)) / (max(y)-min(y))
    
    globals()['norm'+str(i)] = np.column_stack((norm_x, norm_y))
    
    globals()['norm'+str(i)] = globals()['norm'+str(i)].flatten()

In [38]:
test_x = np.zeros((100, len(norm0)))
for i in range(100):
    for j in range(len(norm0)):
        test_x[i][j] = globals()['norm'+str(i)][j]

In [39]:
train_y = pd.read_csv('./dataset/train/train.csv')
train_y = train_y.iloc[:, 22:30]
test_y = pd.read_csv('./dataset/test/test.csv')
test_y = test_y.iloc[:, 22:30]

In [40]:
train_y = train_y.drop(['noseFlatten','nosePronounced','noseBridge','noseCurve'], axis=1)
test_y = test_y.drop(['noseFlatten','nosePronounced','noseBridge','noseCurve'], axis=1)

In [41]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

model = LinearRegression()
model.fit(train_x, train_y)

LinearRegression()

In [42]:
y_pred = model.predict(test_x)
mae = mean_absolute_error(test_y, y_pred)
mae

0.13246269772139338

In [43]:
pred = pd.DataFrame(y_pred)
# pred.columns = ['noseSize','nosePosition', 'noseFlatten','nosePronounced','noseWidth', 'noseBridge', 'noseCurve', 'noseInclination']
pred.columns = ['noseSize', 'nosePosition','noseWidth','noseInclination']
pred

,noseSize,nosePosition,noseWidth,noseInclination
0,0.538307,0.503238,0.242832,0.666046
1,0.844224,0.462933,0.226557,0.480979
2,0.594282,0.319207,0.475643,0.229332
3,0.263803,0.606544,0.374370,0.559612
4,0.459358,0.454127,0.431443,0.666174
...,...,...,...,...
95,0.266050,0.601358,0.294715,0.772653
96,0.446302,0.515655,0.542060,0.487408
97,0.741506,0.524107,0.319460,0.637387
98,0.394396,0.483651,0.605880,0.180990


In [44]:
test_y

,noseSize,nosePosition,noseWidth,noseInclination
0,0.528794,0.450309,0.274802,0.678466
1,0.235096,0.543748,0.544562,0.507280
2,0.400362,0.306366,0.600885,0.239924
3,0.539006,0.705693,0.380126,0.655423
4,0.434991,0.420111,0.600525,0.696418
...,...,...,...,...
95,0.574014,0.662468,0.438891,0.738699
96,0.335086,0.516407,0.402938,0.341421
97,0.377994,0.621198,0.250577,0.682466
98,0.486372,0.390966,0.792901,0.310331
